In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import overpass

In [2]:
df = pd.read_csv(
    '../data/raw/sao-paulo-properties.csv'
)

In [14]:
def get_points_of_interest(lat, lon, radius=500, limit=100):
    api = overpass.API()
    query = f"""
    [out:json];
    node(around:{radius},{lat},{lon})["tourism"];
    out center;
    """
    response = api.get(query)
    #nodes = response.nodes[:limit]
    return response

In [15]:
for index, row in df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    points = get_points_of_interest(lat, lon)
    #df.loc[index, 'poi'] = len(points)

OverpassSyntaxError: [out:json];
    [out:json];
    node(around:500,-23.543138,-46.479486)["tourism"];
    out center;out body;

In [7]:
df['POI'] = df.apply(lambda row: get_poi(row['Latitude'], row['Longitude']), axis=1)

OverpassSyntaxError: [out:json];
    [out:json];
    (
        node(around:5000, -23.543138, -46.479486)["amenity"];
        way(around:5000, -23.543138, -46.479486)["amenity"];
        relation(around:5000, -23.543138, -46.479486)["amenity"];
    );
    out center;
    out body;out body;

In [ ]:
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

In [ ]:
gdf.head()

In [ ]:
def get_nearest_poi(row, poi_gdf):
    distances = poi_gdf.distance(row['geometry'])
    nearest_index = distances.idxmin()
    nearest_poi = poi_gdf.loc[nearest_index]
    return nearest_poi['poi_name'], distances[nearest_index]